In [1]:
import math
import random

import numpy as np
import matplotlib.pyplot as plt

import squigglepy as sq
from squigglepy import bayes
from squigglepy.numbers import K, M, B, T

from copy import copy, deepcopy
from scipy import stats
from pprint import pprint
from tqdm import tqdm
print('Loaded 1')

exec(open('utils.py').read())
print('Loaded 2')

exec(open('modules/tai_timelines.py').read())
print('Loaded TAI timelines module')

exec(open('/Users/peterhurford/dev/forecastflow/library.py').read()) # TODO: Package?
print('Loaded Metaculus module')

Loaded 1
Loaded 2
Loaded TAI timelines module
Loaded Metaculus lib v0.3
Loaded Metaculus module


## AI Timelines Sensitivity Analysis

In [2]:
CURRENT_YEAR = 2023                               # What year to start the run on? (default: 2023)
MAX_YEAR = 2123                                   # What year to end the run on? (default: 2123)
years = list(range(CURRENT_YEAR, MAX_YEAR))

def print_year(y):
    y = y['tai_year']
    return '>{}'.format(MAX_YEAR) if y > MAX_YEAR else str(int(y))


defaults = {'initial_gdp_': 23*T,
            'tai_flop_size_': 31,
            'algo_doubling_rate_': algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=31),
            'possible_algo_reduction_': possible_algo_reduction_fn(min_reduction=2,
                                                                   max_reduction=5,
                                                                   tai_flop_size=31),
            'initial_flop_per_dollar_': 10 ** 18.3,
            'flop_halving_rate_': 2.5,
            'max_flop_per_dollar_': 10 ** 24,
            'initial_pay_': 10 ** 8.2,
            'gdp_growth_': 1.02,
            'max_gdp_frac_': 0.005,
            'willingness_ramp_': 1,
            'spend_doubling_time_': 3,
            'spend_doubling_time_2025_': 1,
            'nonscaling_delay_': {'delay': {'prob': 1, 'length': 4}},
            'willingness_spend_horizon_': 1,
            'print_diagnostic': False,
            'variables': {'CURRENT_YEAR': CURRENT_YEAR, 'MAX_YEAR': MAX_YEAR}}

print('## Default ##')
result = run_tai_model_round(**defaults)
print('{}: {}'.format('Default', print_year(result)))          

## Default ##
Default: 2041


In [3]:
print('## TAI FLOP Size ##')
for t in range(26, 43):
    varset = deepcopy(defaults)
    varset['tai_flop_size_'] = t
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=2, max_speed=3.5, tai_flop_size=t)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=2,
                                                                    max_reduction=5,
                                                                    tai_flop_size=t)
    result = run_tai_model_round(**varset)
    print('FLOP Size for TAI {} log FLOP -> {}'.format(t, print_year(result)))

## TAI FLOP Size ##
FLOP Size for TAI 26 log FLOP -> 2027
FLOP Size for TAI 27 log FLOP -> 2028
FLOP Size for TAI 28 log FLOP -> 2031
FLOP Size for TAI 29 log FLOP -> 2034
FLOP Size for TAI 30 log FLOP -> 2038
FLOP Size for TAI 31 log FLOP -> 2041
FLOP Size for TAI 32 log FLOP -> 2043
FLOP Size for TAI 33 log FLOP -> 2047
FLOP Size for TAI 34 log FLOP -> 2051
FLOP Size for TAI 35 log FLOP -> 2051
FLOP Size for TAI 36 log FLOP -> 2056
FLOP Size for TAI 37 log FLOP -> 2061
FLOP Size for TAI 38 log FLOP -> 2062
FLOP Size for TAI 39 log FLOP -> 2068
FLOP Size for TAI 40 log FLOP -> 2082
FLOP Size for TAI 41 log FLOP -> >2123
FLOP Size for TAI 42 log FLOP -> >2123


In [4]:
print('## GDP Growth ##')
for g in range(0, 5):
    varset = deepcopy(defaults)
    varset['gdp_growth_'] = 1 + (g / 100)
    result = run_tai_model_round(**varset)
    print('GDP Growth Rate {}% -> {}'.format(1 + g / 100, print_year(result)))

## GDP Growth ##
GDP Growth Rate 1.0% -> 2041
GDP Growth Rate 1.01% -> 2041
GDP Growth Rate 1.02% -> 2041
GDP Growth Rate 1.03% -> 2041
GDP Growth Rate 1.04% -> 2041


In [5]:
print('## Max GDP Frac ##')
for g in [1/(500*K), 1/(250*K), 1/(100*K), 1/(50*K), 1/(10*K), 5/(10*K),
          1/2000, 1/1500, 1/1000, 2/1000, 3/1000, 4/1000, 5/1000, 1/100,
          2/100, 3/100, 4/100]:
    varset = deepcopy(defaults)
    varset['max_gdp_frac_'] = g
    result = run_tai_model_round(**varset)
    print('Max GDP Frac {} (1 in ~{:,}) -> {}'.format(round(g, 6), int(round(1 / g)), print_year(result)))

## Max GDP Frac ##
Max GDP Frac 2e-06 (1 in ~500,000) -> 2049
Max GDP Frac 4e-06 (1 in ~250,000) -> 2047
Max GDP Frac 1e-05 (1 in ~100,000) -> 2045
Max GDP Frac 2e-05 (1 in ~50,000) -> 2044
Max GDP Frac 0.0001 (1 in ~10,000) -> 2042
Max GDP Frac 0.0005 (1 in ~2,000) -> 2041
Max GDP Frac 0.0005 (1 in ~2,000) -> 2041
Max GDP Frac 0.000667 (1 in ~1,500) -> 2041
Max GDP Frac 0.001 (1 in ~1,000) -> 2041
Max GDP Frac 0.002 (1 in ~500) -> 2041
Max GDP Frac 0.003 (1 in ~333) -> 2041
Max GDP Frac 0.004 (1 in ~250) -> 2041
Max GDP Frac 0.005 (1 in ~200) -> 2041
Max GDP Frac 0.01 (1 in ~100) -> 2041
Max GDP Frac 0.02 (1 in ~50) -> 2041
Max GDP Frac 0.03 (1 in ~33) -> 2041
Max GDP Frac 0.04 (1 in ~25) -> 2041


In [6]:
print('## Spend Doubling Time ##')
for d in range(0, 51, 2):
    varset = deepcopy(defaults)
    varset['spend_doubling_time_'] = 1 + (d / 10)
    result = run_tai_model_round(**varset)
    print('Spend Doubling Time {}yrs -> {}'.format(1 + d / 10, print_year(result)))

## Spend Doubling Time ##
Spend Doubling Time 1.0yrs -> 2036
Spend Doubling Time 1.2yrs -> 2037
Spend Doubling Time 1.4yrs -> 2038
Spend Doubling Time 1.6yrs -> 2038
Spend Doubling Time 1.8yrs -> 2039
Spend Doubling Time 2.0yrs -> 2039
Spend Doubling Time 2.2yrs -> 2039
Spend Doubling Time 2.4yrs -> 2040
Spend Doubling Time 2.6yrs -> 2040
Spend Doubling Time 2.8yrs -> 2040
Spend Doubling Time 3.0yrs -> 2041
Spend Doubling Time 3.2yrs -> 2041
Spend Doubling Time 3.4yrs -> 2041
Spend Doubling Time 3.6yrs -> 2041
Spend Doubling Time 3.8yrs -> 2041
Spend Doubling Time 4.0yrs -> 2042
Spend Doubling Time 4.2yrs -> 2042
Spend Doubling Time 4.4yrs -> 2042
Spend Doubling Time 4.6yrs -> 2042
Spend Doubling Time 4.8yrs -> 2042
Spend Doubling Time 5.0yrs -> 2042
Spend Doubling Time 5.2yrs -> 2042
Spend Doubling Time 5.4yrs -> 2042
Spend Doubling Time 5.6yrs -> 2043
Spend Doubling Time 5.8yrs -> 2043
Spend Doubling Time 6.0yrs -> 2043


In [7]:
print('## Spend Doubling Time for 2023-2025 ##')
for d in range(5, 31, 1):
    varset = deepcopy(defaults)
    varset['spend_doubling_time_2025_'] = d / 10
    result = run_tai_model_round(**varset)
    print('Spend Doubling Time (2023-2025) {}yrs -> {}'.format(d / 10, print_year(result)))

## Spend Doubling Time for 2023-2025 ##
Spend Doubling Time (2023-2025) 0.5yrs -> 2039
Spend Doubling Time (2023-2025) 0.6yrs -> 2039
Spend Doubling Time (2023-2025) 0.7yrs -> 2040
Spend Doubling Time (2023-2025) 0.8yrs -> 2040
Spend Doubling Time (2023-2025) 0.9yrs -> 2040
Spend Doubling Time (2023-2025) 1.0yrs -> 2041
Spend Doubling Time (2023-2025) 1.1yrs -> 2041
Spend Doubling Time (2023-2025) 1.2yrs -> 2041
Spend Doubling Time (2023-2025) 1.3yrs -> 2041
Spend Doubling Time (2023-2025) 1.4yrs -> 2041
Spend Doubling Time (2023-2025) 1.5yrs -> 2041
Spend Doubling Time (2023-2025) 1.6yrs -> 2041
Spend Doubling Time (2023-2025) 1.7yrs -> 2041
Spend Doubling Time (2023-2025) 1.8yrs -> 2041
Spend Doubling Time (2023-2025) 1.9yrs -> 2041
Spend Doubling Time (2023-2025) 2.0yrs -> 2042
Spend Doubling Time (2023-2025) 2.1yrs -> 2042
Spend Doubling Time (2023-2025) 2.2yrs -> 2042
Spend Doubling Time (2023-2025) 2.3yrs -> 2042
Spend Doubling Time (2023-2025) 2.4yrs -> 2042
Spend Doubling Time 

In [8]:
print('## Initial FLOP per dollar ##')
for d in [17, 17.5, 18, 18.3, 18.5, 19]:
    varset = deepcopy(defaults)
    varset['initial_flop_per_dollar_'] = 10 ** d
    result = run_tai_model_round(**varset)
    print('Initial log FLOP per dollar {} -> {}'.format(d, print_year(result)))
    

## Initial FLOP per dollar ##
Initial log FLOP per dollar 17 -> 2045
Initial log FLOP per dollar 17.5 -> 2043
Initial log FLOP per dollar 18 -> 2042
Initial log FLOP per dollar 18.3 -> 2041
Initial log FLOP per dollar 18.5 -> 2040
Initial log FLOP per dollar 19 -> 2038


In [9]:
print('## Initial pay ##')
for p in range(70, 101, 2):
    varset = deepcopy(defaults)
    varset['initial_pay_'] = 10 ** (p / 10)
    result = run_tai_model_round(**varset)
    print('Initial pay in log 2022$USD {} (~${}) -> {}'.format(p / 10, numerize(10 ** (p / 10)), print_year(result)))

## Initial pay ##
Initial pay in log 2022$USD 7.0 (~$10 million) -> 2045
Initial pay in log 2022$USD 7.2 (~$16 million) -> 2044
Initial pay in log 2022$USD 7.4 (~$25 million) -> 2043
Initial pay in log 2022$USD 7.6 (~$40 million) -> 2043
Initial pay in log 2022$USD 7.8 (~$63 million) -> 2042
Initial pay in log 2022$USD 8.0 (~$100 million) -> 2041
Initial pay in log 2022$USD 8.2 (~$158 million) -> 2041
Initial pay in log 2022$USD 8.4 (~$251 million) -> 2040
Initial pay in log 2022$USD 8.6 (~$398 million) -> 2039
Initial pay in log 2022$USD 8.8 (~$631 million) -> 2039
Initial pay in log 2022$USD 9.0 (~$1 billion) -> 2038
Initial pay in log 2022$USD 9.2 (~$2 billion) -> 2037
Initial pay in log 2022$USD 9.4 (~$3 billion) -> 2037
Initial pay in log 2022$USD 9.6 (~$4 billion) -> 2036
Initial pay in log 2022$USD 9.8 (~$6 billion) -> 2036
Initial pay in log 2022$USD 10.0 (~$10 billion) -> 2035


In [10]:
print('## FLOP halving rate ##')
for f in range(5, 41):
    varset = deepcopy(defaults)
    varset['flop_halving_rate_'] = f / 10
    result = run_tai_model_round(**varset)
    print('FLOP halving rate {} -> {}'.format(f / 10, print_year(result)))

## FLOP halving rate ##
FLOP halving rate 0.5 -> 2033
FLOP halving rate 0.6 -> 2033
FLOP halving rate 0.7 -> 2034
FLOP halving rate 0.8 -> 2035
FLOP halving rate 0.9 -> 2035
FLOP halving rate 1.0 -> 2036
FLOP halving rate 1.1 -> 2036
FLOP halving rate 1.2 -> 2037
FLOP halving rate 1.3 -> 2037
FLOP halving rate 1.4 -> 2037
FLOP halving rate 1.5 -> 2038
FLOP halving rate 1.6 -> 2038
FLOP halving rate 1.7 -> 2039
FLOP halving rate 1.8 -> 2039
FLOP halving rate 1.9 -> 2039
FLOP halving rate 2.0 -> 2039
FLOP halving rate 2.1 -> 2040
FLOP halving rate 2.2 -> 2040
FLOP halving rate 2.3 -> 2040
FLOP halving rate 2.4 -> 2040
FLOP halving rate 2.5 -> 2041
FLOP halving rate 2.6 -> 2041
FLOP halving rate 2.7 -> 2041
FLOP halving rate 2.8 -> 2041
FLOP halving rate 2.9 -> 2041
FLOP halving rate 3.0 -> 2041
FLOP halving rate 3.1 -> 2042
FLOP halving rate 3.2 -> 2042
FLOP halving rate 3.3 -> 2042
FLOP halving rate 3.4 -> 2042
FLOP halving rate 3.5 -> 2042
FLOP halving rate 3.6 -> 2042
FLOP halving rat

In [11]:
print('## Max FLOP per dollar ##')
for f in range(20, 31):
    varset = deepcopy(defaults)
    varset['max_flop_per_dollar_'] = 10 ** f
    result = run_tai_model_round(**varset)
    print('Max log FLOP per 2022USD$1 = {} -> {}'.format(f, print_year(result)))

## Max FLOP per dollar ##
Max log FLOP per 2022USD$1 = 20 -> 2042
Max log FLOP per 2022USD$1 = 21 -> 2041
Max log FLOP per 2022USD$1 = 22 -> 2041
Max log FLOP per 2022USD$1 = 23 -> 2041
Max log FLOP per 2022USD$1 = 24 -> 2041
Max log FLOP per 2022USD$1 = 25 -> 2041
Max log FLOP per 2022USD$1 = 26 -> 2041
Max log FLOP per 2022USD$1 = 27 -> 2041
Max log FLOP per 2022USD$1 = 28 -> 2041
Max log FLOP per 2022USD$1 = 29 -> 2041
Max log FLOP per 2022USD$1 = 30 -> 2041


In [12]:
print('## Algo Doubling Rate Minimum ##')
for m in [1, 1.3, 1.5, 2, 2.5, 3, 3.5]:
    varset = deepcopy(defaults)
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=m, max_speed=3.5, tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Algo doubling rate minimum {} -> {}'.format(m, print_year(result)))

## Algo Doubling Rate Minimum ##
Algo doubling rate minimum 1 -> 2041
Algo doubling rate minimum 1.3 -> 2041
Algo doubling rate minimum 1.5 -> 2041
Algo doubling rate minimum 2 -> 2041
Algo doubling rate minimum 2.5 -> 2041
Algo doubling rate minimum 3 -> 2041
Algo doubling rate minimum 3.5 -> 2041


In [13]:
print('## Algo Doubling Rate Maximum ##')
for m in [2, 2.5, 3, 3.5, 4, 4.5, 5, 6]:
    varset = deepcopy(defaults)
    varset['algo_doubling_rate_'] = algo_halving_fn(min_speed=2, max_speed=m, tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Algo doubling rate maximum {} -> {}'.format(m, print_year(result)))

## Algo Doubling Rate Maximum ##
Algo doubling rate maximum 2 -> 2039
Algo doubling rate maximum 2.5 -> 2039
Algo doubling rate maximum 3 -> 2040
Algo doubling rate maximum 3.5 -> 2041
Algo doubling rate maximum 4 -> 2041
Algo doubling rate maximum 4.5 -> 2042
Algo doubling rate maximum 5 -> 2042
Algo doubling rate maximum 6 -> 2043


In [14]:
print('## Possible Algo Reduction minimum ##')
for m in [0, 1, 2, 3, 4, 5]:
    varset = deepcopy(defaults)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=m,
                                                                    max_reduction=5,
                                                                    tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Possible algo reduction minimum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction minimum ##
Possible algo reduction minimum 0 -> 2046
Possible algo reduction minimum 1 -> 2042
Possible algo reduction minimum 2 -> 2041
Possible algo reduction minimum 3 -> 2040
Possible algo reduction minimum 4 -> 2040
Possible algo reduction minimum 5 -> 2040


In [15]:
print('## Possible Algo Reduction maximum ##')
for m in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    varset = deepcopy(defaults)
    varset['possible_algo_reduction_'] = possible_algo_reduction_fn(min_reduction=2,
                                                                    max_reduction=m,
                                                                    tai_flop_size=defaults['tai_flop_size_'])
    result = run_tai_model_round(**varset)
    print('Possible algo reduction maximum {} -> {}'.format(m, print_year(result)))

## Possible Algo Reduction maximum ##
Possible algo reduction maximum 2 -> 2041
Possible algo reduction maximum 3 -> 2041
Possible algo reduction maximum 4 -> 2041
Possible algo reduction maximum 5 -> 2041
Possible algo reduction maximum 6 -> 2041
Possible algo reduction maximum 7 -> 2041
Possible algo reduction maximum 8 -> 2041
Possible algo reduction maximum 9 -> 2041
Possible algo reduction maximum 10 -> 2041


In [16]:
print('## Willingness ramp ##')
for r in range(1, 11):
    varset = deepcopy(defaults)
    varset['willingness_ramp_'] = r / 10
    result = run_tai_model_round(**varset)
    print('Willingness ramp {}x -> {}'.format(round(10 / r, 1), print_year(result)))

## Willingness ramp ##
Willingness ramp 10.0x -> 2037
Willingness ramp 5.0x -> 2038
Willingness ramp 3.3x -> 2039
Willingness ramp 2.5x -> 2039
Willingness ramp 2.0x -> 2040
Willingness ramp 1.7x -> 2040
Willingness ramp 1.4x -> 2040
Willingness ramp 1.2x -> 2040
Willingness ramp 1.1x -> 2040
Willingness ramp 1.0x -> 2041


In [17]:
print('## Willingness spend horizon ##')
for h in range(1, 11):
    varset = deepcopy(defaults)
    varset['willingness_spend_horizon_'] = h
    result = run_tai_model_round(**varset)
    print('Willingness spend horizon {}yrs -> {}'.format(h, print_year(result)))

## Willingness spend horizon ##
Willingness spend horizon 1yrs -> 2041
Willingness spend horizon 2yrs -> 2041
Willingness spend horizon 3yrs -> 2041
Willingness spend horizon 4yrs -> 2041
Willingness spend horizon 5yrs -> 2041
Willingness spend horizon 6yrs -> 2041
Willingness spend horizon 7yrs -> 2041
Willingness spend horizon 8yrs -> 2041
Willingness spend horizon 9yrs -> 2041
Willingness spend horizon 10yrs -> 2041


## Metaculus

In [18]:
get_question(11558)

11558 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates?
6517 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (February 14, 2023)
Metaculus -
* Min: 3640
* <Min: 0.0
* Q1: 17623.15
* Mid: 51507.61
* Q3: 150542.549
* >Max: 0.029410000000000047
* Max: 2000000
-
6559 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (January 1, 2026)
Me -
* Min: 3640
* <Min: 0
* Q1: 3568581.524
* Mid: 14150587.477
* Q3: 56111686.0
* >Max: 0.027000000000000024
* Max: 1000000000
Metaculus -
* Min: 3640
* <Min: 0.0
* Q1: 618087.295
* Mid: 5889135.777
* Q3: 56111686.0
* >Max: 0.11399999999999999
* Max: 1000000000
-
6192 What will be the maximum compute (in petaFLOPS-days) ever used in training an AI experiment by the following dates? (January 1, 2031)
Me -
* Min: 1800
* <Min: 0
* Q1: 4242640.687
* Mid: 20049751.85

In [19]:
# https://www.metaculus.com/questions/11558/maximum-compute-used-in-ai-training/
# TODO: Fetch from Metaculus, look side by side
# TODO: Be able to predict back
for y in [2022, 2025, 2030]:
    print('-')
    print('## {} ##'.format(y))
    for i in range(3):
        flop_at_max_ = flop_at_max(initial_gdp=defaults['initial_gdp_'],
                                   gdp_growth=[1.02, 1.025, 1.03][i],
                                   initial_pay=[10*M, 100*M, 1*B][i],
                                   spend_doubling_time=[1, 2, 3][i],
                                   max_gdp_frac=[1/1000, 4/1000, 1/100][i],
                                   initial_flop_per_dollar=10 ** 18,
                                   max_flop_per_dollar=10 ** 24,
                                   flop_halving_rate=[2, 2.5, 3][i],
                                   year=(y - CURRENT_YEAR))

        print('{}: {} max log FLOPs / ~{} petaFLOP/s-days)'.format(['25th', 'mean', '75th'][i],
                                                                   np.round(np.log10(flop_at_max_), 1),
                                                                   log_flop_to_petaflop_sdays(np.log10(flop_at_max_))))


-
## 2022 ##
25th: 24.5 max log FLOPs / ~40911 petaFLOP/s-days)
mean: 25.7 max log FLOPs / ~619751 petaFLOP/s-days)
75th: 26.8 max log FLOPs / ~7265380 petaFLOP/s-days)
-
## 2025 ##
25th: 25.9 max log FLOPs / ~924379 petaFLOP/s-days)
mean: 26.5 max log FLOPs / ~4021998 petaFLOP/s-days)
75th: 27.4 max log FLOPs / ~28976286 petaFLOP/s-days)
-
## 2030 ##
25th: 28.1 max log FLOPs / ~159863445 petaFLOP/s-days)
mean: 27.9 max log FLOPs / ~90261450 petaFLOP/s-days)
75th: 28.4 max log FLOPs / ~288816073 petaFLOP/s-days)
